<a href="https://colab.research.google.com/github/Theveetha/Telecom_Churn_Prediction/blob/main/Telecom_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Telecom Churn Prediction***
**GOAL:**
1. Do a detailed analysis to identify and visualize which factors contribute to the customer churn.
2. Perform model prediction for various models and find the best among them.

# **REQUIRED SETUP**
**GOAL**
1. Import the required libraries for our analysis
2. Connect to the Drive to load the dataset
3. Read the multiple csv file and create a single Dataframe

## **Import Required Libraries**

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Set max columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

## **Mount Drive**

In [97]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## **Import the dataset**

In [98]:
churn_data = pd.read_csv('/content/drive/MyDrive/Stats&ML_Dataset/churn_data.csv')
cust_data = pd.read_csv('/content/drive/MyDrive/Stats&ML_Dataset/customer_data.csv')
internet_data = pd.read_csv('/content/drive/MyDrive/Stats&ML_Dataset/internet_data.csv')

# Data Dimension
print(f"Churn has {churn_data.shape[0]} rows and {churn_data.shape[1]} columns")
print(f"Customer has {cust_data.shape[0]} rows and {cust_data.shape[1]} columns")
print(f"Internet has {internet_data.shape[0]} rows and {internet_data.shape[1]} columns")

Churn has 7043 rows and 9 columns
Customer has 7043 rows and 5 columns
Internet has 7043 rows and 9 columns


In [99]:
# Data Columns
for cols in [churn_data,cust_data,internet_data]:
  print(list(cols))

['customerID', 'tenure', 'PhoneService', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']
['customerID', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']


In [100]:
# Data Merge
original_data = pd.merge(churn_data,cust_data,how='inner',on='customerID')
original_data = pd.merge(original_data,internet_data,how='inner',on='customerID')

# Making a copy to work for our analysis
data = original_data.copy()
data.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,Female,0,Yes,No,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No,Male,0,No,No,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Male,0,No,No,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,45,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Male,0,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Female,0,No,No,No,Fiber optic,No,No,No,No,No,No


# **DATA DICTIONARY**
**GOAL**
1. Read the data description to understand the purpose of each variable

In [101]:
desc = pd.read_csv('/content/drive/MyDrive/Stats&ML_Dataset/Telecom_Churn_Data_Dictionary.csv',encoding='latin-1')
desc
# NOTE: Here we are using encoding function as this description file is consisting of some additional languages which needs to be encoded

,S.No.,Variable Name,Meaning
0,1,CustomerID,The unique ID of each customer
1,2,Gender,The gender of a person
2,3,SeniorCitizen,Whether a customer can be classified as a seni...
3,4,Partner,If a customer is married/ in a live-in relatio...
4,5,Dependents,If a customer has dependents (children/ retire...
5,6,Tenure,The time for which a customer has been using t...
6,7,PhoneService,Whether a customer has a landline phone servic...
7,8,MultipleLines,Whether a customer has multiple lines of inter...
8,9,InternetService,The type of internet services chosen by the cu...
9,10,OnlineSecurity,Specifies if a customer has online security.


# **ANALYSING THE FUNDAMENTALS**
**GOAL**
1. Understand the shape and type of the data
2. Analyse the Categorical and Numerical variables
3. Do fundamental data cleaning and data transformation on cases where all its required

## **Dimension**

In [102]:
print(f"Number of Columns : {data.shape[1]}")
print(f"Number of Rows : {data.shape[0]}")

Number of Columns : 21
Number of Rows : 7043


## **Detailed Info on all columns**

In [103]:
# Function for extracting information on all columns
def info(data):
  cols = data.columns
  dtype = data.dtypes
  unique_count = [data[cols].nunique() for cols in data.columns]
  missing_value_count = data.isnull().sum()
  sample = data.head(2).T
  unique_values = [data[cols].unique() if len(data[cols].unique()) <= 5 else 'Unique value count more than 5' for cols in data.columns]

  data_info = pd.DataFrame({'Column_Name':cols
                            ,'Data_Type':dtype
                            ,'Missing_Value_Per':missing_value_count
                            ,'Unique_Value_Count':unique_count
                            ,'Unique_Values':unique_values
                            })
  data_info = pd.concat([data_info,sample],axis=1)
  data_info.rename(columns={0:'Sample_1',1:'Sample_2'},inplace=True)

  data_info.reset_index(drop=True,inplace=True)
  return data_info

info(data)

,Column_Name,Data_Type,Missing_Value_Per,Unique_Value_Count,Unique_Values,Sample_1,Sample_2
0,customerID,object,0,7043,Unique value count more than 5,7590-VHVEG,5575-GNVDE
1,tenure,int64,0,73,Unique value count more than 5,1,34
2,PhoneService,object,0,2,"[No, Yes]",No,Yes
3,Contract,object,0,3,"[Month-to-month, One year, Two year]",Month-to-month,One year
4,PaperlessBilling,object,0,2,"[Yes, No]",Yes,No
5,PaymentMethod,object,0,4,"[Electronic check, Mailed check, Bank transfer...",Electronic check,Mailed check
6,MonthlyCharges,float64,0,1585,Unique value count more than 5,29.85,56.95
7,TotalCharges,object,0,6531,Unique value count more than 5,29.85,1889.5
8,Churn,object,0,2,"[No, Yes]",No,No
9,gender,object,0,2,"[Female, Male]",Female,Male


## **Numerical and Categorical Columns**

In [104]:
num_cols = [cols for cols in data.columns if data[cols].dtype != 'O']
cat_cols = [cols for cols in data.columns if data[cols].dtype == 'O']
print(f"Number of Numerical Columns : {len(num_cols)}")
print(f"Number of Categorical Columns : {len(cat_cols)}")

Number of Numerical Columns : 3
Number of Categorical Columns : 18


## **Duplicate Rows**

In [105]:
count = data[data.duplicated()].shape
print(f"There are no duplicates in the dataset" if count[0]==0 else f"There are {count[0]} duplicates in the dataset")

There are no duplicates in the dataset


## **OBSERVATION**
1. TotalCharges should be typecasted to float type as currently its of object type
2. CustomerId has all values to be unique, but we are not deleting it as we will be using it for future analysis
3. We can see that there are no columns with single unique values
4. All the variables have 0 missing value
5. There are no duplicate records in the dataset

## **Data Cleaning - Type Casting**

In [106]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'],errors='coerce')
data['TotalCharges'].dtypes

dtype('float64')

In [108]:
num_cols = [cols for cols in data.columns if data[cols].dtype != 'O']
cat_cols = [cols for cols in data.columns if data[cols].dtype == 'O']
print(f"Number of Numerical Columns : {len(num_cols)}")
print(f"Number of Categorical Columns : {len(cat_cols)}")

Number of Numerical Columns : 4
Number of Categorical Columns : 17


## **Five Number Summary**

In [109]:
data[num_cols].describe()

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen
count,7043.000000,7043.000000,7032.000000,7043.000000
mean,32.371149,64.761692,2283.300441,0.162147
std,24.559481,30.090047,2266.771362,0.368612
min,0.000000,18.250000,18.800000,0.000000
25%,9.000000,35.500000,401.450000,0.000000
50%,29.000000,70.350000,1397.475000,0.000000
75%,55.000000,89.850000,3794.737500,0.000000
max,72.000000,118.750000,8684.800000,1.000000


In [110]:
data[cat_cols].describe()

,customerID,PhoneService,Contract,PaperlessBilling,PaymentMethod,Churn,gender,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,3,2,4,2,2,2,2,3,3,3,3,3,3,3,3
top,7590-VHVEG,Yes,Month-to-month,Yes,Electronic check,No,Male,No,No,No,Fiber optic,No,No,No,No,No,No
freq,1,6361,3875,4171,2365,5174,3555,3641,4933,3390,3096,3498,3088,3095,3473,2810,2785


## **OBSERVATION**
1. Post Type casting there are 11 missing values in the TotalCharges column
2. Since the difference between mean and median is less than 10% of 1-SD, replace the missing values with Median

## **Data Manipulation - Handle Missing Value**